# Binary classificator problem

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression

In [2]:
train = pd.read_csv('../Data/train.csv', index_col=0)
test = pd.read_csv('../Data/test.csv', index_col=0)

display(train.head())

test.head()

,FRAUDE,VALOR,HORA_AUX,Dist_max_NAL,Canal1,FECHA,COD_PAIS,CANAL,DIASEM,DIAMES,...,INGRESOS,EGRESOS,NROPAISES,Dist_Sum_INTER,Dist_Mean_INTER,Dist_Max_INTER,NROCIUDADES,Dist_Mean_NAL,Dist_HOY,Dist_sum_NAL
id,,,,,,,,,,,,,,,,,,,,,
9000000001,1,0.0,13,659.13,ATM_INT,20150501,US,ATM_INT,5,1,...,1200000.0,1200000.0,1,NaN,NaN,NaN,6,474.94,4552.41,5224.36
9000000002,1,0.0,17,594.77,ATM_INT,20150515,US,ATM_INT,5,15,...,5643700.0,500000.0,1,NaN,NaN,NaN,5,289.99,4552.41,2029.90
9000000003,1,0.0,13,659.13,ATM_INT,20150501,US,ATM_INT,5,1,...,1200000.0,1200000.0,1,NaN,NaN,NaN,6,474.94,4552.41,5224.36
9000000004,1,0.0,13,659.13,ATM_INT,20150501,US,ATM_INT,5,1,...,1200000.0,1200000.0,1,NaN,NaN,NaN,6,474.94,4552.41,5224.36
9000000005,1,0.0,0,1.00,ATM_INT,20150510,CR,ATM_INT,0,10,...,0.0,0.0,1,NaN,NaN,NaN,1,NaN,1482.35,1.00


,FRAUDE,VALOR,HORA_AUX,Dist_max_COL,Dist_max_INTER,Canal1,FECHA_FRAUDE,COD_PAIS,CANAL,FECHA,...,Dist_Mean_INTER,Dist_Max_INTER,NROCIUDADES,Dist_Sum_NAL,Dist_Mean_NAL,Dist_HOY,Dist_sum_NAL,Dist_mean_NAL,Dist_sum_INTER,Dist_mean_INTER
id,,,,,,,,,,,,,,,,,,,,,
98523068,NaN,42230.09,18,1.00,1.00,POS,20150515,US,POS,20150515,...,NaN,NaN,1,NaN,NaN,4552.41,1.00,1.00,1.00,1.00
300237898,NaN,143202.65,20,614.04,7632.97,POS,20150506,US,MCI,20150506,...,6092.69,7632.97,2,1228.07,614.04,4552.41,1228.07,614.04,24370.75,6092.69
943273308,NaN,243591.25,2,286.84,2443.14,ATM_INT,20150517,EC,ATM_INT,20150517,...,1743.52,2443.14,7,1944.35,138.88,5083.41,1944.35,138.88,6974.09,1743.52
951645809,NaN,238267.40,20,1.00,1.00,ATM_INT,20150508,EC,ATM_INT,20150508,...,NaN,NaN,1,NaN,NaN,904.81,1.00,1.00,1.00,1.00
963797516,NaN,490403.58,13,1.00,1.00,ATM_INT,20150501,US,ATM_INT,20150501,...,NaN,NaN,1,NaN,NaN,4552.41,1.00,1.00,1.00,1.00


In [3]:
y_train = train['FRAUDE']
X_train = train.drop('FRAUDE', axis=1)

X_test = test.drop('FRAUDE', axis=1)

In [4]:
train_columns = list(X_train.columns)
test_columns = list(X_test.columns)

# Extract columns in common
common_columns = list(set(train_columns) & set(test_columns))

# Extract columns in train but not in test and viceversa
train_not_test = list(set(train_columns) - set(test_columns))
test_not_train = list(set(test_columns) - set(train_columns))

In [5]:
print('Columns in common: ', common_columns)
print('Columns in train but not in test: ', train_not_test)
print('Columns in test but not in train: ', test_not_train)

Columns in common:  ['Dist_Mean_NAL', 'FECHA_VIN', 'Dist_Mean_INTER', 'VALOR', 'NROCIUDADES', 'INGRESOS', 'Dist_Max_INTER', 'DIASEM', 'NROPAISES', 'SEGMENTO', 'Canal1', 'OFICINA_VIN', 'EGRESOS', 'SEXO', 'EDAD', 'COD_PAIS', 'Dist_sum_NAL', 'FECHA', 'DIAMES', 'HORA_AUX', 'Dist_Sum_INTER', 'Dist_HOY', 'CANAL']
Columns in train but not in test:  ['Dist_max_NAL']
Columns in test but not in train:  ['Dist_max_COL', 'Dist_max_INTER', 'Dist_Sum_NAL', 'Dist_sum_INTER', 'Dist_mean_INTER', 'Dist_mean_NAL', 'FECHA_FRAUDE']


In [9]:
from fuzzywuzzy import fuzz
from collections import defaultdict

def find_similar_columns(df, threshold=92):
    # Create a dictionary where the keys are lower-case column names
    # and the values are lists of original column names that map to the lower-case column name
    column_dict = defaultdict(list)

    cols = list(df.columns)
    while len(cols) > 0:
        col = cols.pop(0)
        column_dict[col].append(col)
        for other_col in cols:
            if fuzz.ratio(col.lower(), other_col.lower()) >= threshold:
                column_dict[col].append(other_col)
        cols = [x for x in cols if x not in column_dict[col]]

    # Filter out entries in the dictionary where there are not at least two similar column names
    similar_columns = {key: value for key, value in column_dict.items() if len(value) > 1}
    
    return similar_columns


In [13]:
X_test[["FECHA", "FECHA_FRAUDE"]]
X_test["FECHA"].equals(X_test["FECHA_FRAUDE"])

True

In [15]:
# Test the function
similar_columns = find_similar_columns(X_test)

# Add another pair of columns by hand
similar_columns['fecha'] = ['FECHA', 'FECHA_FRAUDE']

print(similar_columns)

{'Dist_max_INTER': ['Dist_max_INTER', 'Dist_Max_INTER'], 'Dist_Sum_INTER': ['Dist_Sum_INTER', 'Dist_sum_INTER'], 'Dist_Mean_INTER': ['Dist_Mean_INTER', 'Dist_mean_INTER'], 'Dist_Sum_NAL': ['Dist_Sum_NAL', 'Dist_sum_NAL'], 'Dist_Mean_NAL': ['Dist_Mean_NAL', 'Dist_mean_NAL'], 'fecha': ['FECHA', 'FECHA_FRAUDE']}


In [11]:
# Test the function
similar_columns = find_similar_columns(X_train)
print(similar_columns)

{}


In [32]:
# Function to investigate possible duplicated columns in a dataframe
def investigate_similar_columns(df: pd.DataFrame, first_column: str, second_column: str ):
    """ Function to investigate possible duplicated columns in a dataframe
    
    Args:
        df (pd.DataFrame): Dataframe to investigate
        first_column (str): Name of the first column to compare
        second_column (str): Name of the second column to compare

    Returns:
        None
    """
    # Compare the two columns in the previous line
    condition = df[first_column] == df[second_column]

    # Check if non-null values are the same
    non_null_values_same = df[condition][first_column].equals(df[condition][second_column])

    print('Non-null values are the same: ', non_null_values_same)

    # Display different records
    different_records = df[~condition][[first_column, second_column]]

    # Boolean variable that indicates that there is at least one null value in the first column
    first_column_null = df[first_column].isnull().sum() > 0

    # Boolean variable that indicates that there is at least one null value in the second column
    second_column_null = df[second_column].isnull().sum() > 0

    # Check the boolean variables and print if there is a null value in any of the columns or in both
    # If only one column has all non-null values, print the name of the column and a message saying
    # This column has all non-null values. Keeping this column and dropping the other one.
    if first_column_null and second_column_null:
        print('Both columns have null values')
    elif first_column_null:
        print('Column with all non-null values: ', second_column)
        print('Keeping this column and dropping the other one: ', first_column)
    elif second_column_null:
        print('Column with all non-null values: ', first_column)
        print('Keeping this column and dropping the other one: ', second_column)
    else:
        print('Both columns have all non-null values')

In [19]:
# Investigating X_test
X_test[["Dist_sum_NAL", "Dist_Sum_NAL"]]

# Compare the two columns in the previous line
condition = X_test["Dist_sum_NAL"] == X_test["Dist_Sum_NAL"]

X_test[~condition][["Dist_sum_NAL", "Dist_Sum_NAL"]]

# Check if non-null values are the same
X_test[condition]["Dist_sum_NAL"].equals(X_test[condition]["Dist_Sum_NAL"])
# Result is true. Keeping column with imputed values

#X_test[condition][["Dist_sum_NAL", "Dist_Sum_NAL"]]

True

In [20]:
# In X_test check if "FECHA" and "FECHA_FRAUDE" are the same equal
X_test["FECHA"].equals(X_test["FECHA_FRAUDE"])
# Result is true. Keeping column "FECHA"

True

In [27]:
# Investigating X_test on Dist_sum_INTER
#X_test[["Dist_Sum_INTER", "Dist_sum_INTER"]]

# Compare the two columns in the previous line
condition = X_test["Dist_Sum_INTER"] == X_test["Dist_sum_INTER"]

#X_test[~condition][["Dist_Sum_INTER", "Dist_sum_INTER"]]
# Result: "Dist_sum_INTER" has imputed values.

# Check if non-null values are the same
#X_test[condition]["Dist_Sum_INTER"].equals(X_test[condition]["Dist_sum_INTER"])
# Result is true. Keeping column with imputed values

X_test[condition][["Dist_Sum_INTER", "Dist_sum_INTER"]]

,Dist_Sum_INTER,Dist_sum_INTER
id,,
300237898,24370.75,24370.75
943273308,6974.09,6974.09
971691350,20435.63,20435.63
971788614,115833.16,115833.16
978266544,15715.64,15715.64
979598220,9104.82,9104.82
982775975,12959.36,12959.36
994412379,18209.64,18209.64
998545844,4552.41,4552.41


In [31]:
# Investigating X_test on Dist_Mean_INTER
#X_test[["Dist_Mean_INTER", "Dist_mean_INTER"]]

# Compare the two columns in the previous line
condition = X_test["Dist_Mean_INTER"] == X_test["Dist_mean_INTER"]

#X_test[~condition][["Dist_Mean_INTER", "Dist_mean_INTER"]]
# Result: "Dist_mean_INTER" has imputed values. Uncommon column

# Check if non-null values are the same
#X_test[condition]["Dist_Mean_INTER"].equals(X_test[condition]["Dist_mean_INTER"])
# Result is true. Keeping column with imputed values

X_test[condition][["Dist_Mean_INTER", "Dist_mean_INTER"]]

,Dist_Mean_INTER,Dist_mean_INTER
id,,
300237898,6092.69,6092.69
943273308,1743.52,1743.52
971691350,2919.38,2919.38
971788614,5036.22,5036.22
978266544,5238.55,5238.55
979598220,4552.41,4552.41
982775975,4319.79,4319.79
994412379,4552.41,4552.41
998545844,4552.41,4552.41


In [ ]:
# Investigating X_test on Dist_Mean_INTER
#X_test[["Dist_Mean_INTER", "Dist_mean_INTER"]]

# Compare the two columns in the previous line
condition = X_test["Dist_Mean_INTER"] == X_test["Dist_mean_INTER"]

#X_test[~condition][["Dist_Mean_INTER", "Dist_mean_INTER"]]
# Result: "Dist_mean_INTER" has imputed values. Uncommon column

# Check if non-null values are the same
#X_test[condition]["Dist_Mean_INTER"].equals(X_test[condition]["Dist_mean_INTER"])
# Result is true. Keeping column with imputed values

X_test[condition][["Dist_Mean_INTER", "Dist_mean_INTER"]]

In [6]:
X_train = X_train[common_columns]
X_test = X_test[common_columns]

In [7]:
X_train.head()

,DIASEM,EGRESOS,NROCIUDADES,Dist_Mean_NAL,COD_PAIS,EDAD,HORA_AUX,Canal1,NROPAISES,Dist_Sum_INTER,...,INGRESOS,SEGMENTO,Dist_Mean_INTER,CANAL,VALOR,OFICINA_VIN,DIAMES,Dist_HOY,FECHA_VIN,SEXO
id,,,,,,,,,,,,,,,,,,,,,
9000000001,5,1200000.0,6,474.94,US,29.0,13,ATM_INT,1,NaN,...,1200000.0,Personal Plus,NaN,ATM_INT,0.0,392.0,1,4552.41,20120306.0,M
9000000002,5,500000.0,5,289.99,US,29.0,17,ATM_INT,1,NaN,...,5643700.0,Personal Plus,NaN,ATM_INT,0.0,716.0,15,4552.41,20050415.0,M
9000000003,5,1200000.0,6,474.94,US,29.0,13,ATM_INT,1,NaN,...,1200000.0,Personal Plus,NaN,ATM_INT,0.0,392.0,1,4552.41,20120306.0,M
9000000004,5,1200000.0,6,474.94,US,29.0,13,ATM_INT,1,NaN,...,1200000.0,Personal Plus,NaN,ATM_INT,0.0,392.0,1,4552.41,20120306.0,M
9000000005,0,0.0,1,NaN,CR,25.0,0,ATM_INT,1,NaN,...,0.0,Personal,NaN,ATM_INT,0.0,788.0,10,1482.35,20141009.0,M


In [8]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2965 entries, 9000000001 to 1953178702
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   DIASEM           2965 non-null   int64  
 1   EGRESOS          2941 non-null   float64
 2   NROCIUDADES      2965 non-null   int64  
 3   Dist_Mean_NAL    2508 non-null   float64
 4   COD_PAIS         2965 non-null   object 
 5   EDAD             2941 non-null   float64
 6   HORA_AUX         2965 non-null   int64  
 7   Canal1           2965 non-null   object 
 8   NROPAISES        2965 non-null   int64  
 9   Dist_Sum_INTER   1418 non-null   float64
 10  Dist_sum_NAL     2965 non-null   float64
 11  FECHA            2965 non-null   int64  
 12  Dist_Max_INTER   1418 non-null   float64
 13  INGRESOS         2941 non-null   float64
 14  SEGMENTO         2941 non-null   object 
 15  Dist_Mean_INTER  1418 non-null   float64
 16  CANAL            2965 non-null   object 
 17  VALO

In [9]:
X_train[['Dist_sum_NAL','Dist_Mean_NAL']]

,Dist_sum_NAL,Dist_Mean_NAL
id,,
9000000001,5224.36,474.94
9000000002,2029.90,289.99
9000000003,5224.36,474.94
9000000004,5224.36,474.94
9000000005,1.00,NaN
...,...,...
622529101,484.30,96.86
2043206272,2810.75,82.67
2943206272,2810.75,82.67


In [1]:
from xgboost import XGBRegressor